In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from typing import List
import pandas as pd
import numpy as np
import json
import warnings
from langchain_core.messages import HumanMessage, SystemMessage
pd.set_option('display.max_columns', None)

In [2]:
llm = ChatOpenAI(base_url="http://THOTH.local:1234/v1/", api_key="na", temperature=0.5)

In [3]:
datagen_template = '''
Generate an artificial medical research dataset containing 30 columns.
The subject of this dataset is: {subject}.

Generate a total of 100 rows and return the dataset as a CSV object.
'''
datagen_prompt = PromptTemplate(
    template=datagen_template,
    input_variables=["subject"],
)
datagen_chain = datagen_prompt | llm
datagen_output = datagen_chain.invoke({"subject": "age at first stroke"})


In [4]:
from io import StringIO

In [5]:
#extract the CSV data from the blueprint
resp = datagen_output.content
#extract the part of the string surrounded by ```csv
csv_start = resp.find("```csv")
csv_end = resp.find("```", csv_start+1)
csv_data = resp[csv_start+6:csv_end]

In [43]:
#parse the CSV data into a pandas dataframe
#raw_data = pd.read_csv(StringIO(csv_data))
raw_data.head()

,ID,Age at First Stroke (years),Gender,Hypertension,Diabetes Mellitus,Hyperlipidemia,Smoking Status,Alcohol Consumption,BMI,Physical Activity,Family History of Stroke,Stroke Type,Blood Pressure (mmHg),Cholesterol (mg/dL),Glucose (mg/dL),Creatinine (mg/dL),Uric Acid (mg/dL),Albumin (g/L),Globulin (g/L),Total Protein (g/L),White Blood Cell Count (x10^9/L),Platelet Count (x10^9/L),Red Blood Cell Count (x10^12/L),Hemoglobin (g/dL),Hematocrit (%),Mean Corpuscular Volume (fL),Platelet Distribution Width (%),Prolactin (μIU/mL),Insulin-like Growth Factor-1 (ng/mL),Tumor Necrosis Factor-alpha (pg/mL),Interleukin-6 (pg/mL),C-reactive Protein (mg/L),Fibrinogen (g/L)
0,1,65,Male,Yes,No,Yes,Current Smoker,Moderate,24.3,Regular,No,Ischemic Stroke,130/85,210,120,0.9,5.2,38,25,66,6.8,246,4.7,14.1,13.5,34,15.2,96,19.5,2.3,NaN,NaN,NaN
1,2,72,Female,Yes,No,No,Ex-smoker,NaN,28.9,Sedentary,Yes,Hemorrhagic Stroke,140/90,220,110,1.0,6.4,35,27,68,7.1,230,4.5,13.8,12.2,36,16.5,100,21.3,2.9,NaN,NaN,NaN
2,3,58,Male,No,Yes,No,Current Smoker,Heavy,26.2,Regular,No,Ischemic Stroke,120/80,200,100,0.8,4.1,40,28,64,6.5,250,4.6,14.3,13.7,34,15.9,90,18.2,2.1,NaN,NaN,NaN
3,4,68,Female,Yes,No,Yes,Ex-smoker,Moderate,29.8,Sedentary,Yes,Hemorrhagic Stroke,150/95,230,130,1.1,5.6,37,26,70,7.2,210,4.4,13.9,12.3,38,17.1,105,20.8,2.7,NaN,NaN,NaN
4,5,63,Male,Yes,No,No,Current Smoker,Heavy,25.5,Regular,No,Ischemic Stroke,130/85,210,115,0.9,4.9,39,29,67,6.8,240,4.7,14.2,13.1,36,16.3,95,19.1,2.5,NaN,NaN,NaN


In [6]:
import numpy as np
import scipy.stats as stats

def best_cont_dist(col):
    col = col.dropna()  # Remove any missing values
    col = col.astype(float)  # Convert to float
    # Calculate basic statistics
    mean = np.mean(col)
    median = np.median(col)
    std_dev = np.std(col)
    
    # Fit different distributions and find the best fit
    distributions = [
        stats.uniform,  # Uniform distribution
        stats.norm,  # Normal distribution
        stats.expon,  # Exponential distribution
        stats.gamma,  # Gamma distribution
        stats.lognorm,  # Log-normal distribution
        stats.beta,  # Beta distribution
    ]
    
    best_fit = None
    best_fit_name = ""
    best_fit_params = ()
    best_fit_error = np.inf
    
    for distribution in distributions:
        try:
            # Fit the distribution to the data
            params = distribution.fit(col)
            
            # Calculate the error between the fitted distribution and the data
            error = stats.kstest(col, distribution.name, args=params).statistic
            
            # Update the best fit if the error is lower
            if error < best_fit_error:
                best_fit = distribution
                best_fit_name = distribution.name
                best_fit_params = params
                best_fit_error = error
        except Exception as e:
            print(f"Error fitting {distribution.name}: {e}")
    
    return {
        "best_fit_distribution": best_fit_name,
        "best_fit_params": best_fit_params,
        "best_fit_error": best_fit_error
    }

In [7]:
#create a function which summarizes every column in a pandas dataframe (both numeric and categorical) and returns a json array which can then be used by a large language model to select relevant columns for answering a specific question
def summarize_dataframe(df):
    summary = []
    
    # Iterate over each column in the dataframe
    for column in df.columns:
        column_summary = {}
        column_summary['name'] = column
        column_summary['type'] = str(df[column].dtype)

        # Check if the column is boolean
        if df[column].dtype == 'bool':
            column_summary['summary'] = df[column].value_counts().to_dict()
            column_summary["dist"] = "Binary"                

        # Check if the column is numeric
        elif pd.api.types.is_numeric_dtype(df[column]):
            column_summary['summary'] = df[column].describe().to_dict()
            with warnings.catch_warnings():
                warnings.simplefilter('ignore')
                column_summary["dist"] = best_cont_dist(df[column])
                        
        else:
            column_summary['summary'] = df[column].value_counts().to_dict()
            column_summary["dist"] = None
        
        summary.append(column_summary)
    
    return summary
summary = summarize_dataframe(raw_data)
summary

Error fitting uniform: zero-size array to reduction operation minimum which has no identity
Error fitting norm: attempt to get argmax of an empty sequence
Error fitting expon: zero-size array to reduction operation minimum which has no identity
Error fitting gamma: zero-size array to reduction operation minimum which has no identity
Error fitting lognorm: zero-size array to reduction operation minimum which has no identity
Error fitting beta: zero-size array to reduction operation minimum which has no identity
Error fitting uniform: zero-size array to reduction operation minimum which has no identity
Error fitting norm: attempt to get argmax of an empty sequence
Error fitting expon: zero-size array to reduction operation minimum which has no identity
Error fitting gamma: zero-size array to reduction operation minimum which has no identity
Error fitting lognorm: zero-size array to reduction operation minimum which has no identity
Error fitting beta: zero-size array to reduction operatio

[{'name': 'ID',
  'type': 'int64',
  'summary': {'count': 30.0,
   'mean': 15.5,
   'std': 8.803408430829505,
   'min': 1.0,
   '25%': 8.25,
   '50%': 15.5,
   '75%': 22.75,
   'max': 30.0},
  'dist': {'best_fit_distribution': 'uniform',
   'best_fit_params': (1.0, 29.0),
   'best_fit_error': 0.03333333333333333}},
 {'name': 'Age at First Stroke (years)',
  'type': 'int64',
  'summary': {'count': 30.0,
   'mean': 66.46666666666667,
   'std': 5.745963100649272,
   'min': 58.0,
   '25%': 61.25,
   '50%': 66.5,
   '75%': 71.0,
   'max': 75.0},
  'dist': {'best_fit_distribution': 'beta',
   'best_fit_params': (0.5142003981252006,
    0.4682386557986524,
    57.85330302229336,
    17.146696977706643),
   'best_fit_error': 0.1344303899757423}},
 {'name': 'Gender',
  'type': 'object',
  'summary': {'Male': 15, 'Female': 15},
  'dist': None},
 {'name': 'Hypertension',
  'type': 'object',
  'summary': {'Yes': 16, 'No': 14},
  'dist': None},
 {'name': 'Diabetes Mellitus',
  'type': 'object',
  '

In [8]:
class DataGuruSchema(BaseModel):
    predictor_cols: List[str] = Field(description="Dataset columns which will be used to predict the outcome variables.")
    output_cols: List[str] = Field(description="Dataset columns which will be predicted by the input columns.")
    excluded_cols: List[str] = Field(description="Dataset columns which will not be used in the model.")

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=DataGuruSchema)

In [9]:
def parser_to_prompt_schema(parser):
    schema = {k: v for k, v in parser._get_schema(parser.pydantic_object).items()}

    # Remove extraneous fields.
    reduced_schema = schema
    if "title" in reduced_schema:
        del reduced_schema["title"]
    if "type" in reduced_schema:
        del reduced_schema["type"]
    # Ensure json in context is well-formed with double quotes.
    schema_str = json.dumps(reduced_schema)

    return schema_str

In [67]:
class ResearchQuestionSchema(BaseModel):
    question: str = Field(description="The research question to be answered by the model.")
    outcome: str = Field(description="A single outcome column from the dataset to be predicted by the model.")
    predictors: List[str] = Field(description="The input columns to be used to predict the outcome column. Try to include as many columns as possible to improve the model's accuracy.")
    model_type: List[str] = Field(description="The sklearn model types to be used for prediction. May include multiple to find the best option. Options include: 'LinearRegression', 'LogisticRegression', 'RandomForestClassifier', 'RandomForestRegressor', 'GradientBoostingClassifier', 'GradientBoostingRegressor', 'SVC', 'KNeighborsClassifier', 'KNeighborsRegressor', 'DecisionTreeClassifier', 'DecisionTreeRegressor', etc")
class ResearcherSchema(BaseModel):
    topics: List[ResearchQuestionSchema] = Field(description="A list of research questions to be answered by the model.")
# Set up a parser + inject instructions into the prompt template.
researcher_parser = JsonOutputParser(pydantic_object=ResearcherSchema)

In [74]:
summary

[{'name': 'ID',
  'type': 'int64',
  'summary': {'count': 30.0,
   'mean': 15.5,
   'std': 8.803408430829505,
   'min': 1.0,
   '25%': 8.25,
   '50%': 15.5,
   '75%': 22.75,
   'max': 30.0},
  'dist': {'best_fit_distribution': 'uniform',
   'best_fit_params': (1.0, 29.0),
   'best_fit_error': 0.03333333333333333}},
 {'name': 'Age at First Stroke (years)',
  'type': 'int64',
  'summary': {'count': 30.0,
   'mean': 66.46666666666667,
   'std': 5.745963100649272,
   'min': 58.0,
   '25%': 61.25,
   '50%': 66.5,
   '75%': 71.0,
   'max': 75.0},
  'dist': {'best_fit_distribution': 'beta',
   'best_fit_params': (0.5142003981252006,
    0.4682386557986524,
    57.85330302229336,
    17.146696977706643),
   'best_fit_error': 0.1344303899757423}},
 {'name': 'Gender',
  'type': 'object',
  'summary': {'Male': 15, 'Female': 15},
  'dist': None},
 {'name': 'Hypertension',
  'type': 'object',
  'summary': {'Yes': 16, 'No': 14},
  'dist': None},
 {'name': 'Diabetes Mellitus',
  'type': 'object',
  '

In [70]:
researcher_template = '''
You are a helpful assistant that answers in JSON.

Your task is to review a summary of a dataset and develop a list of 3-5 research questions to be answered using the available data.

Below is a summary of the dataset in JSON format:
{summary}

Here's the json schema you must adhere to for your response:
<schema>
{schema}
</schema>
'''

researcher_prompt = PromptTemplate(
    template=researcher_template,
    input_variables=["subject", "summary"],
    partial_variables={"schema": parser_to_prompt_schema(researcher_parser)},
)

researcher_chain = researcher_prompt | llm | parser
researcher_output = researcher_chain.invoke({"summary": json.dumps({"columns": summary})})
display(researcher_output)

{'topics': [{'question': 'What factors are associated with a higher risk of stroke?',
   'outcome': 'Age at First Stroke (years)',
   'predictors': ['Hypertension',
    'Diabetes Mellitus',
    'Hyperlipidemia',
    'Smoking Status',
    'Alcohol Consumption',
    'BMI',
    'Physical Activity',
    'Family History of Stroke'],
   'model_type': ['LogisticRegression']},
  {'question': 'Can we predict the likelihood of a patient having hypertension based on their demographic and health data?',
   'outcome': 'Hypertension',
   'predictors': ['Age at First Stroke (years)',
    'Gender',
    'BMI',
    'Cholesterol (mg/dL)',
    'Glucose (mg/dL)',
    'Creatinine (mg/dL)',
    'Uric Acid (mg/dL)',
    'Albumin (g/L)',
    'Globulin (g/L)',
    'Total Protein (g/L)'],
   'model_type': ['LogisticRegression']},
  {'question': 'Is there a correlation between blood pressure and the risk of stroke?',
   'outcome': 'Stroke Type',
   'predictors': ['Blood Pressure (mmHg)',
    'Age at First Stroke 

In [73]:
reasearch_results = []

for topic in researcher_output["topics"]:
    print(f"Research Question: {topic['question']}")
    print(f"Outcome: {topic['outcome']}")
    print(f"Predictors: {topic['predictors']}")

    clean_data = raw_data.dropna(subset=topic["predictors"] + [topic["outcome"]])
    
    X = clean_data[topic["predictors"]]
    y = clean_data[topic["outcome"]]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    if pd.api.types.is_numeric_dtype(y):
        print("Generating a predictor for a continuous value")
        ebm = ExplainableBoostingRegressor()
        ebm.fit(X_train, y_train)
        print("Score:", ebm.score(X_test, y_test))
        reasearch_results.append({"topic": topic, "model": ebm})
    else:
        ebm = ExplainableBoostingClassifier()
        ebm.fit(X_train, y_train)
        print("Score:", ebm.score(X_test, y_test))
        reasearch_results.append({"topic": topic, "model": ebm})

Research Question: What factors are associated with a higher risk of stroke?
Outcome: Age at First Stroke (years)
Predictors: ['Hypertension', 'Diabetes Mellitus', 'Hyperlipidemia', 'Smoking Status', 'Alcohol Consumption', 'BMI', 'Physical Activity', 'Family History of Stroke']
Generating a predictor for a continuous value
Score: 0.31441057450689547
Research Question: Can we predict the likelihood of a patient having hypertension based on their demographic and health data?
Outcome: Hypertension
Predictors: ['Age at First Stroke (years)', 'Gender', 'BMI', 'Cholesterol (mg/dL)', 'Glucose (mg/dL)', 'Creatinine (mg/dL)', 'Uric Acid (mg/dL)', 'Albumin (g/L)', 'Globulin (g/L)', 'Total Protein (g/L)']
Score: 1.0
Research Question: Is there a correlation between blood pressure and the risk of stroke?
Outcome: Stroke Type
Predictors: ['Blood Pressure (mmHg)', 'Age at First Stroke (years)', 'Gender']
Score: 1.0
Research Question: Can we predict the likelihood of a patient having diabetes based o

In [29]:
dataguru_template = '''
You are a helpful assistant that answers in JSON.

Your task is to review a summary of a dataset and return which columns should be included as predictor and outcome variables in a machine learning model analysis.
Columns that are irrelevant should be excluded.
There may be more than one outcome column.
All columns must be categorized into one of the three categories: predictor, outcome, or excluded. Each column must be assigned to only one of these categories.

The subject of your investigation is: {subject}.

Below is a summary of the dataset in JSON format:
{summary}

Here's the json schema you must adhere to for your response:
<schema>
{schema}
</schema>
'''

dataguru_prompt = PromptTemplate(
    template=dataguru_template,
    input_variables=["subject", "summary"],
    partial_variables={"schema": parser_to_prompt_schema(parser)},
)

dataguru_chain = dataguru_prompt | llm | parser
dataguru_ouptut = dataguru_chain.invoke({"subject": "age at first stroke", "summary": json.dumps({"columns": summary})})
dataguru_ouptut


{'input_cols': ['Age at First Stroke (years)',
  'BMI',
  'Blood Pressure (mmHg)',
  'Cholesterol (mg/dL)',
  'Creatinine (mg/dL)',
  'Glucose (mg/dL)',
  'Uric Acid (mg/dL)',
  'White Blood Cell Count (x10^9/L)',
  'Platelet Count (x10^9/L)',
  'Red Blood Cell Count (x10^12/L)',
  'Hemoglobin (g/dL)',
  'Hematocrit (%)',
  'Mean Corpuscular Volume (fL)',
  'Platelet Distribution Width (%)',
  'Prolactin (μIU/mL)',
  'Insulin-like Growth Factor-1 (ng/mL)',
  'Tumor Necrosis Factor-alpha (pg/mL)'],
 'output_cols': ['Stroke Type'],
 'excluded_cols': ['ID',
  'Gender',
  'Hypertension',
  'Diabetes Mellitus',
  'Hyperlipidemia',
  'Smoking Status',
  'Alcohol Consumption',
  'Physical Activity',
  'Family History of Stroke',
  'Albumin (g/L)',
  'Globulin (g/L)',
  'Total Protein (g/L)']}

In [31]:
X = raw_data[dataguru_ouptut["input_cols"]]
y = raw_data[dataguru_ouptut["output_cols"][0]]

In [32]:
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from interpret.glassbox import ExplainableBoostingClassifier, ExplainableBoostingRegressor
from interpret import show

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

if pd.api.types.is_numeric_dtype(y):
    ebm = ExplainableBoostingRegressor()
    ebm.fit(X_train, y_train)
else:
    ebm = ExplainableBoostingClassifier()
    ebm.fit(X_train, y_train)
    print("Score:", ebm.score(X_test, y_test))

Score: 1.0


In [34]:
ebm.score(X_train, y_train)

1.0

In [35]:
show(ebm.explain_global())